Name of Members:

- __APA, GIUSIPPI MARIA II DEL ROSARIO__
- __OBEJERO, REY FERBS MAGALLON__
- __RAMIREZ, BENMAR SIM GREFALDA__
- __RAMOS, RONN PATRICK BICERA__

## I. Introduction

The dataset we chose for this machine project is the "Sports" dataset. 

Our project will have us go through the process of selecting a dataset, describing it, performing EDA, data preprocessing and cleaning, model training, HP tuning, model selection, and extracting insights from the data.

We aim to leverage the data from the soccer matches to preduct the number of goals that will be scored during the remaining playtime. The target of this project is regression as we aim to predict a continous outcome, which is the **number of goals to be scored** during the remaining time.

We will implement the use of machine learning and other techniques taught in class to build our predictive model.

## II. Description

The dataset is related to soccer matches and will be used to predict how many goals will be scored during the remaining match time. The dataset contains 10,000 such snapshots, each taken at a different, random point in time during a match. 

In the context of this dataset, a “snapshot” refers to a set of data points captured at a specific moment during a soccer match.

As for the collection process, it can be inferred that the data might have been collected in real-time during this soccer match, capturing various events and statistics at different timestamps. This data would allow for more accurate predictions of future events based on the current state of the match, and whether it is best to bet on over or under a certain number of goals.

There are only 7000 instances(rows) and 21 features(columns) in the dataset. Each of the features are statistics of both events for the home and away team.

### List of Variables

| **Variable Name** | **Description**|
|--------------------------------------|----------------|
|**uuid** | Snapshot’s unique ID|
|**current_minute** | Match’s current minute, including 15 min half-time|
|**home_score and away_score** | Home and away team’s goals.|
|**home_yellow_cards and away_yellow_cards**| Home and away team’s yellow cards.|
|**home_red_cards and away_red_cards**| Home and away team’s red cards.|
|**home_attacks and away_attacks**| Home and away team’s attempted attacks.|
|**home_dangerous_attacks and away_dangerous_attacks**| Home and away team’s dangerous attacks.|
|**home_corners and away_corners**| Home and away team’s awarded corners.|
|**home_off_target and away_off_target**| Home and away team’s off-target shots.|
|**home_on_target and away_on_target**| Home and away team’s on-target shots.|
|**home_possession and away_possession** |Home and away team’s ball possession %.|

## III. Modules

For this machine project, we will utilize the following Python libraries:

- `one`:
- `two`:
- `three`:
- `four`:

In [ ]:
from linear_regression import compute_RMSE
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

## IV. Data Cleaning

We will now be performing various methods of preprocessing and data cleaning to enhance the dataset's usability and suitability to our exploratory analysis and model training.

This is also to ensure that the dataset will be ready for use, which means we will be checking for missing values, duplicates, outliers and other errors. Making it so that the data is in its correct representation.

In [ ]:
# Importing and reading the Sports.csv
sports = pd.read_csv("sports.csv")
sports.head()

**Checking for Missing Values**



The code here indentifies and prints the columns in our sports Dataframe that contain missing values, first one counts the total number of missing values in each column, and if the count is greater than zero it shows us the list.

This is an essential step, just so we can see if we have any columns that are missing data before we move on to other processes.

In [ ]:
# Check each column for missing values
missing_values = sports.isnull().sum()

columns_with_missing_values = missing_values[missing_values > 0]
print(columns_with_missing_values)

**Checking for Duplicates**



This code identifies and removes duplicates rows in the sports DataFrame, it finds all the duplicate rows and seperates them and prints out the rows. Duplicates are removed from the original dataframe and will only keep the first occurence. 

In [ ]:
# Find the duplicates
duplicate_uuids = sports.duplicated(keep=False)

# Create a dataframe for duplicates
duplicates = sports[duplicate_uuids]

# Display the rows with duplicates
print("Duplicate Entries:\n", duplicates)

# Removing the duplicates if detected in dataframe
if not duplicates.empty:
    sports = sports.drop_duplicates(keep='first')
    print("Duplicates removed. Dataset has {sports.shape[0]} rows.")
else:
    print("No duplicates found.")

**Other Cleaning Processes**

The first process replaces infinite values with NaN, which is for undefined data. Drops any columns where there are more than 50% values are missing, while the last checks for inconsistencies in the possession data, wherein it looks for home_possession and way_possession variables that do not equate to 100.

In [ ]:
# Replacing any erroneous infinite values
sports = sports.replace([np.inf, -np.inf], np.nan)

# Dropping columns with more than 50% missing values
sports = sports.dropna(thresh=len(sports)*0.5, axis=1)

# Check for incorrect possession percentages
incorrect_possession_indices = sports[~(sports['home_possession'] + sports['away_possession'] == 100)].index

if len(incorrect_possession_indices) > 0:
    print("Found inconsistencies in possession data:")
    print(sports.loc[incorrect_possession_indices, ['home_possession', 'away_possession']])
else:
    print("Possession data is consistent.")

As you can see there are percentages that don't equate to 100, but the difference is neglible because it is only from 99 to 100 which won't cause much noise in the analysis. It isn't exactly necessary to remove it at this point of the dataset but is also another consideration we took to prepare the if ever the dataset grew larger.

**Data Type Checking**

Having appropriate data types for the values in the dataset is crucial to data analysis and model training. Luckily for us, the dataset we currently have has the appropriate values as all of the variables and their values are appropriate to their data type.

The importance of having correct data types is that we have the following:

Data Integrity,
Accuracy,
Efficiency,
Compatibility, 
and Reliability

In [ ]:
# Printing all of the present data types
print("All data types present:\n", sports.dtypes)

### Outlier Detection and Treatment

Outliers can significantly impact statistical measures and model performance. In the context of our dataset, outliers might represent unusual game conditions, errors in data collection, or truly exceptional events. 

Surprisingly enough, there are no major outliers in our dataset, because in the dataset, varying values like attacks, cards, and accuracy are all subject to the team's playstyle and ability. 

**Our Approach**

But we performed winsorization on the dataset to make it robust against potential outliers in the future if ever there are any in the future, possibly when the dataset grows larger. By capping the data at specified percentiles we reduce the possible influed of these outliers. 

We also provided the before and after outlier treatment visualizations to identify the outliers (if there are any).

In [ ]:
# Columns to check for outliers
outlier_columns = ['current_minute', 'home_yellow_cards', 'away_yellow_cards', 'home_on_target',
                   'away_on_target', 'home_attacks', 'away_attacks', 'home_dangerous_attacks',
                   'away_dangerous_attacks', 'home_corners', 'away_corners', 'home_off_target',
                   'away_off_target', 'home_possession', 'away_possession']

def winsorize(data, lower_percentile=5, upper_percentile=95):
    # Winsorizes data by capping values at specified percentiles
    lower_bound = np.percentile(data, lower_percentile)
    upper_bound = np.percentile(data, upper_percentile)
    return np.clip(data, lower_bound, upper_bound)

def truncate(data, upper_bound):
    # Truncates data by removing values exceeding the upper bound
    return data[data <= upper_bound]

def detect_possession_outliers(data, method='zscore', z_score_threshold=3, lower_bound=20, upper_bound=80):
    home_possession = data['home_possession']
    away_possession = data['away_possession']

    if method == 'zscore':
        # Z-score method (not used for normalization)
        pass
    elif method == 'percentile':
        # Percentile-based method (not used for normalization)
        pass
    elif method == 'domain':
        # Domain knowledge-based method
        data['home_possession_outlier'] = np.where((home_possession < lower_bound) | (home_possession > upper_bound), True, False)
        data['away_possession_outlier'] = np.where((away_possession < lower_bound) | (away_possession > upper_bound), True, False)
    else:
        raise ValueError("Invalid outlier detection method. Choose 'zscore', 'percentile', or 'domain'.")

    return data

# Handle outliers for other columns
for col in outlier_columns:
    if col == 'current_minute':
        sports[col] = truncate(sports[col], 105)
    elif col.endswith('_cards'):
        sports[col] = winsorize(sports[col], upper_percentile=80)
    else:
        sports[col] = winsorize(sports[col])

# Choose a method for possession outliers and apply it
sports = detect_possession_outliers(sports.copy(), method='domain', lower_bound=20, upper_bound=80)  # Example: using domain-based method

# Calculate goals scored after current minute
def calculate_remaining_goals(row):
    total_goals = row['home_score'] + row['away_score']
    goals_so_far = total_goals - row['home_score'] - row['away_score']
    return total_goals - goals_so_far

sports['goals_remaining'] = sports.apply(calculate_remaining_goals, axis=1)

# Calculate additional features
epsilon = 1e-10
sports['home_attack_efficiency'] = sports['home_score'] / (sports['home_attacks'] + epsilon)
sports['away_attack_efficiency'] = sports['away_score'] / (sports['away_attacks'] + epsilon)

sports['home_shot_conversion_rate'] = sports['home_score'] / (sports['home_on_target'] + sports['home_off_target'] + epsilon)
sports['away_shot_conversion_rate'] = sports['away_score'] / (sports['away_on_target'] + sports['away_off_target'] + epsilon)

sports['possession_differential'] = sports['home_possession'] - sports['away_possession']

# Select columns for normalization (excluding uuid)
columns_to_normalize = [col for col in sports.columns if col != 'uuid']

# Create a MinMaxScaler object (normalizes to 0 to 1 range)
scaler = MinMaxScaler(feature_range=(0, 1))

# Apply normalization to selected columns
sports[columns_to_normalize] = scaler.fit_transform(sports[columns_to_normalize])

# Visualize distributions after outlier treatment and normalization
fig, axes = plt.subplots(nrows=len(outlier_columns) + 2, ncols=2, figsize=(15, 40))

for i, col in enumerate(outlier_columns):
    # Box plots
    sns.boxplot(data=sports, x=col, ax=axes[i, 0])
    axes[i, 0].set_title(f'{col} - Before Treatment')

    sns.boxplot(data=sports, x=col, ax=axes[i, 1])
    axes[i, 1].set_title(f'{col} - After Treatment')

# Visualize possession data with outliers
sns.boxplot(data=sports, x='home_possession', ax=axes[len(outlier_columns), 0], hue='home_possession_outlier')
sns.boxplot(data=sports, x='away_possession', ax=axes[len(outlier_columns), 1], hue='away_possession_outlier')

# Histograms for possession data with outliers
sns.histplot(data=sports, x='home_possession', kde=True, ax=axes[len(outlier_columns) + 1, 0], hue='home_possession_outlier')
sns.histplot(data=sports, x='away_possession', kde=True, ax=axes[len(outlier_columns) + 1, 1], hue='away_possession_outlier')

plt.tight_layout()
plt.show()

We did boxplots and histograms for all relevant variables and utilized truncation and winsorization. 

In our current case, there are no obvious outliers, because the initial data might not have contained extreme values that would significantly impact the results. There is also the idea of playstyle variations where the teams at different current_minutes would have varying possessions, attacks, or accuracy.

Despite this, still applying outlier detection methods still offers a safety net from future outliners. We also applied normalization to bring the features to a common scale between 0 and 1, this will help us ensure that there is feature scaling and improved algorithm performance from algorithms like such as neural networks and gradient descent.

### Data Transformation

**Feature Engineering**
`goals_remaining`: This is our target variable, representing the number of goals scored in the remaining time of the match.

Ratio of Attack to Goals

`home_attack_efficiency`: Measures the efficiency of the home team in converting attacks into goals.
`away_attack_efficiency`: Measures the efficiency of the away team in converting attacks into goals.

Ratio of Shots to Goals

`home_shot_conversion_rate`: Measures the efficiency of the home team in converting shots into goals.
`away_shot_conversion_rate`: Measures the efficiency of the away team in converting shots into goals.

In [ ]:
# Filter out rows where current_minute is less than or equal to 0
sports = sports[sports['current_minute'] > 0]

# Calculate goals scored after current minute
def calculate_potential_goals(row):
    remaining_time = 105 - row['current_minute']
    goals_so_far = row['home_score'] + row['away_score']
    avg_goals_per_minute = goals_so_far / row['current_minute']
    potential_goals = avg_goals_per_minute * remaining_time
    return potential_goals

sports['potential_goals'] = sports.apply(calculate_potential_goals, axis=1)

# Filter out rows where the sum of home and away attacks and dangerous attacks is zero
sports = sports[(sports['home_attacks'] + sports['home_dangerous_attacks'] > 0) & 
                (sports['away_attacks'] + sports['away_dangerous_attacks'] > 0)]

# Filter out rows where the sum of home and away shots is zero
sports = sports[(sports['home_on_target'] + sports['home_off_target'] > 0) & 
                (sports['away_on_target'] + sports['away_off_target'] > 0)]

# Calculate additional features
sports['home_attack_efficiency'] = sports['home_score'] / (sports['home_attacks'] + sports['home_dangerous_attacks'])
sports['away_attack_efficiency'] = sports['away_score'] / (sports['away_attacks'] + sports['away_dangerous_attacks'])
sports['home_shot_conversion_rate'] = sports['home_score'] / (sports['home_on_target'] + sports['home_off_target'])
sports['away_shot_conversion_rate'] = sports['away_score'] / (sports['away_on_target'] + sports['away_off_target'])

# Select new features for normalization
new_features_to_normalize = ['potential_goals', 'home_attack_efficiency', 'away_attack_efficiency', 
                             'home_shot_conversion_rate', 'away_shot_conversion_rate']

# Create a MinMaxScaler object (normalizes to 0-1 range)
scaler = MinMaxScaler()

# Apply normalization to new features
sports[new_features_to_normalize] = scaler.fit_transform(sports[new_features_to_normalize])

print(sports['potential_goals'].head())


**Dropping Features**

Some data from the additional features were divided by zero therefore creating infinite values. The uuid column is a unique identifier that does not provide any predictive information, therefore it's correct to drop these.

In [ ]:
# Dropping irrelevant column
sports = sports.drop('uuid', axis=1)

# Replace infinite values with NaN
sports.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
sports.dropna(inplace=True)

**Correlation Matrix**

A **correlation matrix** is a table showing the correlation coefficients between many variables. Each entry in the table shows the correlation between two variables. A correlation coefficient is a statistical measure that calculates the strength of the relationship between the relative movements of the two variables. The range of values for the correlation coefficient is -1.0 to 1.0. If a correlation is close to 1, it indicates a strong positive relationship between the variables. When that number is close to -1, the variables have a strong negative relationship.

In the given heatmap:

- Dark blue cells indicate a strong positive correlation, dark red cells indicate a strong negative correlation, and white cells indicate no correlation.

- For example, 'home_score' and 'home_attack_efficiency' have a strong positive correlation. This suggests that as the home team's score increases, their attack efficiency also tends to increase.

- Conversely, 'away_possession_outlier' and 'home_attack_efficiency' have a strong negative correlation, shown by a darker red shade and numerical value close to -1. This suggests that when the away team's possession is considered an outlier (either extremely high or low), the home team's attack efficiency tends to be lower.


In [ ]:
corr_matrix = sports.corr()

plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix of Sports DataFrame')
plt.show()

**Removal of Features**

We decided to remove the card-related variables as they might not have a direct impact on specific metrics, and are focused on which specific players do get these cards, which is data not present. Attack efficiency and ratios of goals to shots has more of a direct influence to the number of goals.

If the dataset did have the extra data for individual players, we believe that focusing more on the attack-related variables would give us a clearer answer as these unnecessary variables can add noise into our model and possibly lead to overfitting.

In [ ]:
columns_to_drop = ['home_yellow_cards', 'away_yellow_cards', 'home_red_cards', 'away_red_cards']
sports = sports.drop(columns=columns_to_drop)

## Exploratory Data Analysis

EDA is a critical step in any data analysis or machine learning project. It allows us to understand the data’s characteristics, identify patterns and relationships, detect anomalies, inform feature engineering, and check assumptions. For instance, EDA on the ‘Sports’ dataset will provide insights into the factors influencing the number of goals scored during the remaining playtime. These insights will guide our choice of regression model and help us engineer features that improve our model’s predictive performance. 

We can try to look at the average goals remaining for all the games, to have a general idea of it.

In [ ]:
average_potential_goals = sports['potential_goals'].mean()

print(f"Average Potential Goals: {average_potential_goals}")

Here we will generate a history plot of the goals_remaining variable to visualize its distribution.

In [ ]:
sns.histplot(sports['potential_goals'], kde=True)
plt.title('Distribution of Potential Goals')
plt.show()

Right-skewness for goals remaining indicates that while most matches have fewer goals remaining, there are some matches where a significantly higher number of goals are scored towards the end. This distribution suggests that matches can vary widely in the number of goals scored in the remaining time.

Below we can first visualize the home attack efficiency and shot conversion rate, here in the Kernel Density Estimate (KDE), it's basically a smoothed out version of the histogram so we can better see the distribution and probability density.

In [ ]:
sns.histplot(sports['home_attack_efficiency'], kde=True)
plt.title('Distribution of Home Attack Efficiency')
plt.xlim(0, 1)  
plt.show()

sns.histplot(sports['home_shot_conversion_rate'], kde=True)
plt.title('Distribution of Home Shot Conversion Rate')
plt.xlim(0, 1) 
plt.show()

Here we can generate the away attack efficiency and away shot conversion rate. We see a significant difference in their KDE as well.

In [ ]:
sns.histplot(sports['away_attack_efficiency'], kde=True)
plt.title('Distribution of Away Attack Efficiency')
plt.xlim(0, 1) 
plt.show()

sns.histplot(sports['away_shot_conversion_rate'], kde=True)
plt.title('Distribution of Away Shot Conversion Rate')
plt.xlim(0, 1) 
plt.show()

The features attack_efficiency and shot_conversion_rate for both teams are right-skewed. This means that most teams have lower values for these features, indicating that they are generally less efficient in converting attacks into goals. However, there are a few teams with very high values, suggesting that only a small number of teams achieve high efficiency in these areas.


In [ ]:
sns.scatterplot(x='home_attack_efficiency', y='potential_goals', data=sports)
plt.title('Home Attack Efficiency vs. Goals Remaining')
plt.xlabel('Home Attack Efficiency')
plt.ylabel('Potential Goals')
plt.show()

sns.scatterplot(x='home_shot_conversion_rate', y='potential_goals', data=sports)
plt.title('Home Shot Conversion Rate vs. Goals Remaining')
plt.xlabel('Shot Conversion Rate')
plt.ylabel('Potential Goals')
plt.show()

Since the goals remaining vary despite most teams having a low attack efficiency means that this variabile could be due to other factors influencing the number of goals remaining that are not captured by attack efficiency and shot conversion alone.

In [ ]:
sns.scatterplot(x='away_attack_efficiency', y='potential_goals', data=sports)
plt.title('Away Attack Efficiency vs. Potential Goals')
plt.xlabel('Home Attack Efficiency')
plt.ylabel('Potential Goals')
plt.show()


sns.scatterplot(x='away_shot_conversion_rate', y='potential_goals', data=sports)
plt.title('Away Shot Conversion Rate vs. Potential Goals')
plt.xlabel('Shot Conversion Rate')
plt.ylabel('Potential Goals')
plt.show()

The same interpretation can be concluded with the away attack efficiency, but it now differs with the away shot conversion rate where the points are much more scattered but is still mostly right-skewed.

## Model Training

Model training is a crucial step in the machine learning pipeline. It involves learning the underlying patterns in the training data so that we can make predictions on unseen data. The goal is to find a model that generalizes well, meaning it accurately predicts the outcomes on new, unseen data based on the patterns it learned from the training data.

In this project, we trained several types of regression models on our 'Sports' dataset:

1. **Linear Regression**: This is a simple and commonly used type of regression that assumes a linear relationship between the input variables (x) and the single output variable (y). It can be used when the relationship between the input and output variables is approximately linear.

2. **Regression Trees (Decision Trees)**: This is a type of model that breaks down our dataset into smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes, which provide a clear interpretation of why the model is making certain predictions.

3. **Polynomial Regression**: This is a type of regression that models the relationship between the input variable (x) and the output variable (y) as an nth degree polynomial. Polynomial regression can model relationships between variables that aren't linear and can fit data with curves or slopes.

### Linear Regression

Getting the model ready for the data.

In [ ]:
X_sports = sports['home_attacks'].values.reshape(-1, 1)
y_sports = sports['home_dangerous_attacks'].values

X_train, X_test, y_train, y_test = train_test_split(X_sports, y_sports, test_size=0.20, random_state=0)

print('Training data shape:', X_train.shape)
print('Ground truth values shape:', y_train.shape)

Visualizing the data.

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_train, y_train, 'ro', label='Original data')

plt.ylabel('Home dangerous attacks')
plt.xlabel('Home attacks')
ax.legend()
plt.grid()
plt.show()

Train the model using gradient descent.

In [ ]:
model = SGDRegressor(eta0=1e-9, max_iter=200, learning_rate='optimal', random_state=1, verbose=1, alpha=0.001)

model.fit(X_train, y_train)

Show the predicted linear regression line.

In [ ]:
# Plot the model
fig, ax = plt.subplots()
ax.plot(X_train, y_train, 'ro', label='Original data')

y_predicted = model.predict(X_train)
line, = ax.plot(X_train, y_predicted, label='Regression line')

ax.legend()
plt.grid()
plt.show()

Show accuracy using RMSE

In [ ]:
rmse = compute_RMSE(y_train, y_predicted)
print('{:.2f}'.format(rmse))

Try on the test set.

In [ ]:
# Plot the model
fig, ax = plt.subplots()
ax.plot(X_test, y_test, 'ro', label='Original data')

y_predicted = model.predict(X_test)
line, = ax.plot(X_test, y_predicted, label='Regression line')

ax.legend()
plt.grid()
plt.show()

Accuracy on the test set.

In [ ]:
rmse = compute_RMSE(y_test, y_predicted)
print('{:.2f}'.format(rmse))

#### Error Analysis

Error analysis is a critical step in the machine learning pipeline, especially in regression models. It helps us understand how well our model is performing and where it’s falling short. By analyzing the residuals (the differences between the predicted and actual values), we can identify patterns that the model may have missed.

Making a residual plot for error analysis

In [ ]:
residuals = y_test - y_predicted  # Calculate residuals

# Scatter plot of residuals vs. predicted values
plt.scatter(y_predicted, residuals)  
plt.xlabel("Predicted Home Dangerous Attacks")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.grid(True)
plt.show()

The chart above is a scatter plot of the residuals (the differences between the actual and the predicted values against the predicted values). Here we have a well performing model because the residuals are randomy scattered around at zero, with no clear pattern.

This means that the model is correctly specified and that the error term in the underlying regression equation has a constant variance, suggesting that the model's predictions are fairly accurate.

#### Printing all evaluation metrics

Printing all evaluation metrics provides a comprehensive understanding of a model's performance. Each metric offers unique insights into different aspects of the model's predictions.

In [ ]:
# Calculate additional metrics (R-squared)
r2 = r2_score(y_test, y_predicted)

# Print all evaluation metrics
print("Mean Squared Error:", mean_squared_error(y_test, y_predicted))
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_predicted)))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_predicted))
print("R-squared:", r2)

The MSE is relatively low which means the model's predictions are faily close to the actual values. 

RMSE means that the predictions are only 0.1375 units away from actual.

MAE indicates there are no strong influences to the RMSE

R Squared means there is 73.96% of the variability in the output variable.

### Bagging
Get the data ready for training.

In [ ]:
X_sports = sports[['home_shot_conversion_rate',                   
                  'away_shot_conversion_rate',
                  'home_attack_efficiency',
                  'away_attack_efficiency',
                  'current_minute']].values
y_sports = sports['potential_goals'].values

X_train, X_test, y_train, y_test = train_test_split(X_sports, y_sports, test_size=0.20, random_state=0)

print('Training data shape:', X_train.shape)
print('Ground truth values shape:', y_train.shape)

Train the model.

In [ ]:
model = BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=7), random_state = 0, n_estimators=7)

model.fit(X_train, y_train)

Accuracy of the model.

In [ ]:
print(model.score(X_train, y_train))

Accuracy on the test set.

In [ ]:
print(model.score(X_test, y_test))

#### Error Analysis and printing all evaluation metrics

Making a residual plot for error analysis also printed all evaluation metrics (MSE, MAE, R Squared, and RMSE)

In [ ]:
# Predicting the values
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculating residuals
train_residuals = y_train - y_train_pred
test_residuals = y_test - y_test_pred

# Plotting residuals
plt.figure(figsize=(10,6))
plt.scatter(y_train_pred, train_residuals, c='blue', marker='o', label='Training data')
plt.scatter(y_test_pred, test_residuals, c='lightgreen', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=0, xmax=1, color='red')
plt.show()

# Printing evaluation metrics
print("Training Metrics:")
print("Mean Squared Error: ", mean_squared_error(y_train, y_train_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_train, y_train_pred))
print("R^2 Score: ", r2_score(y_train, y_train_pred))
print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_train, y_train_pred)))

print("\nTest Metrics:")
print("Mean Squared Error: ", mean_squared_error(y_test, y_test_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_test_pred))
print("R^2 Score: ", r2_score(y_test, y_test_pred))
print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_test, y_test_pred)))

From the residuals it represents the difference between the actual values and the predicted values. Here we can see that the residuals are somewhat scattered around 0 but there is a slight pattern.

The training residuals seem to be more concentrated around the zero which may mean that there is a slight overfitting of the training data.

The MSE and RMSE are really low meaning there are only small errors in the model's prediction.

There is strong correlation since the R Squared scores are high.

### Polynomial Regression

Set the y value to the target variable and X to the features.

In [ ]:
X = sports[['home_attack_efficiency',
            'away_attack_efficiency', 
            'home_shot_conversion_rate', 
            'away_shot_conversion_rate',
            'current_minute']]

y = sports['potential_goals']

Apply polynomial features transformation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

Train the model

In [ ]:
model = LinearRegression()
model.fit(X_train_poly, y_train)

Make predictions on the test set

In [ ]:
y_pred = model.predict(X_test_poly)

Evaluate the model

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

The MSE is high which means the model's predictions are far from the actual. 

RMSE means that the predictions are 1.2868 units away from actual.

The resulting evalutation tells us that the model is not performing well

## Improving Model

Linear Regression

In [ ]:
# Define the parameter grid for tuning
param_grid = {
    'eta0': [1e-3, 1e-4, 1e-5],  # Learning rate
    'alpha': [0.0001, 0.001, 0.01],  # L1 regularization parameter
}

# Create a GridSearchCV object
grid_search = GridSearchCV(SGDRegressor(random_state=1), param_grid, cv=5, scoring='r2')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best model for prediction
best_model = grid_search.best_estimator_

# Predict on train and test sets using the best model
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate evaluation metrics
train_rmse = compute_RMSE(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

test_rmse = compute_RMSE(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print all evaluation metrics
print("Train Metrics:")
print("  Mean Squared Error:", train_mse)
print("  Root Mean Squared Error:", train_rmse)
print("  Mean Absolute Error:", train_mae)
print("  R-squared:", train_r2)

print("\nTest Metrics:")
print("  Mean Squared Error:", test_mse)
print("  Root Mean Squared Error:", test_rmse)
print("  Mean Absolute Error:", test_mae)
print("  R-squared:", test_r2)

#### These are the results:

- Best Parameters: {'alpha': 0.0001, 'eta0': 0.001}
- Best Score: 0.07001159914785912

**Train Metrics:**
- Mean Squared Error: 0.024995654090193435
- Root Mean Squared Error: 0.15810013943761542
- Mean Absolute Error: 0.12633720728724668
- R-squared: 0.07746293106517332

Bagging Regression

In [ ]:
# Define hyperparameter grid
param_grid = {
    'estimator__max_depth': [3, 5, 7],  
    'n_estimators': [50, 100, 150],  
}

# Create Bagging Regressor with DecisionTreeRegressor base estimator
model = BaggingRegressor(estimator=DecisionTreeRegressor(random_state=0), random_state=0)

# Perform grid search with R-squared scoring
grid_search = GridSearchCV(model, param_grid, scoring='r2', cv=5)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Train the model with best parameters on entire training set
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate on training and test sets with best model
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

train_residuals = y_train - y_train_pred
test_residuals = y_test - y_test_pred

# Print evaluation metrics with best model
print("\nTrain Metrics:")
print("Mean Squared Error:", mean_squared_error(y_train, y_train_pred))
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("Mean Absolute Error:", mean_absolute_error(y_train, y_train_pred))
print("R-squared:", r2_score(y_train, y_train_pred))

print("\nTest Metrics:")
print("Mean Squared Error:", mean_squared_error(y_test, y_test_pred))
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_test_pred))
print("R-squared:", r2_score(y_test, y_test_pred))

#### These are the results:

- Best Parameters: {'estimator__max_depth': 7, 'n_estimators': 150}
- Best Score: 0.873743163527551

**Train Metrics:**
- Mean Squared Error: 0.002218637296984191
- Root Mean Squared Error: 0.04710241285734937
- Mean Absolute Error: 0.025711594505075968
- R-squared: 0.9181147594056243

Polynomial Regression

In [ ]:
# Create a pipeline without scaling
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('regressor', LinearRegression())
])

# Define the hyperparameter grid
param_distributions = {
    'poly__degree': [1, 2, 3, 4, 5],  # Degrees of polynomial features
    'regressor__fit_intercept': [True, False]  # Whether to calculate the intercept
}

# Perform randomized search (using 'r2' for hyperparameter tuning)
random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=5, scoring='r2', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters:", random_search.best_params_)
print("Best R-squared score:", random_search.best_score_)

# Evaluate the best model on train and test sets
best_model = random_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate evaluation metrics for train and test sets
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print all evaluation metrics
print("\nTrain Metrics:")
print("  Mean Squared Error:", train_mse)
print("  Root Mean Squared Error:", train_rmse)
print("  Mean Absolute Error:", train_mae)
print("  R-squared:", train_r2)

print("\nTest Metrics:")
print("  Mean Squared Error:", test_mse)
print("  Root Mean Squared Error:", test_rmse)
print("  Mean Absolute Error:", test_mae)
print("  R-squared:", test_r2)


#### These are the results:

- Best parameters: {'regressor__fit_intercept': False, 'poly__degree': 2}
- Best R-squared score: 0.8322812324337265

**Train Metrics:**
- Mean Squared Error: 0.0037156122363760525
- Root Mean Squared Error: 0.060955822005580834
- Mean Absolute Error: 0.038374204271194604
- R-squared: 0.8628645599960688

Best parameters: {'regressor__fit_intercept': False, 'poly__degree': 2}

Best R-squared score: 0.8322812324337348

Test MSE: 1.6558916980757519

Test RMSE: 1.2868145546564789

Test R-squared: -61.81931414615804


## Model Selection

## Insights and Conclusions

## Reference